In [14]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np

In [15]:
limefeat = pd.read_csv("limefeat.csv")
logregfeat = pd.read_csv("logregfeat.csv")

In [16]:
def featparse(df):
    df['record_type'] = [item.split("_")[0] for item in df.feature]
    df['unit'] = [item.split("_")[1] for item in df.feature]
    df['num_agg'] = [item.split("_")[-1] for item in df.feature]

    feat_var = []
    temp_agg = []
    
    for i in df.index:
        featstr = df.feature.loc[i]
        if df.record_type.loc[i]=='ocnd':
            feat_var.append(featstr.split("_")[2])
            temp_agg.append(None)
        else:
            feat_var.append(featstr.split("_")[3])
            temp_agg.append(featstr.split("_")[2])
    
    df['feat_var'] = feat_var
    df['temp_agg'] = temp_agg

In [17]:
df = pd.read_csv("lime-robustness-all-score-outcome.csv")
feats = []
for i in df.index:
    tmp = df.unq.loc[i]
    tmp = tmp.rstrip("}")
    tmp = tmp.lstrip("{")
    feats.append(tmp.split(","))
feats = [item for sublist in feats for item in sublist]
feats = [item.translate(None," '") for item in feats]

#### Record Type prevalence overall vs prevalence in LIME models

In [18]:
fi = pd.read_csv("32092_2014-04-11_feature-importances.csv")
fi = fi[['feature','feature_importance']]
fi['feature_importance_abs'] = fi.feature_importance.apply(abs)
fi.sort_values(by="feature_importance_abs",ascending=False,inplace=True)
fi.reset_index(drop=True,inplace=True)
featparse(fi)

In [19]:
limefeatlist = set(limefeat.feature)
logregfeatlist = set(logregfeat.feature)

In [20]:
fc = pd.DataFrame({"feature":[item[0] for item in Counter(feats).most_common()],
              "frequency":[item[1] for item in Counter(feats).most_common()]})

featparse(fc)
tmp = pd.DataFrame({"feature":list(logregfeatlist-limefeatlist),
                    "frequency":0})
featparse(tmp)
fc = fc.append(tmp)
fc.reset_index(drop=True,inplace=True)

In [21]:
tmp = fc[['record_type','frequency']]
tmp = tmp.groupby("record_type").agg(sum)
tmp.reset_index(inplace=True)

In [22]:
temp = pd.DataFrame(fc.record_type.value_counts())
temp.reset_index(inplace=True)
temp.rename(columns={"record_type":"overall_freq",
                    "index":"record_type"},
           inplace=True)

In [23]:
temp.set_index("record_type",inplace=True)
tmp = tmp.join(temp,on="record_type")

In [43]:
tmp.to_csv("freq.csv",index=False)